In [1]:
from requests_html import HTMLSession
import pandas as pd

url = '	https://tmpfiles.org/dl/234075/product-list-craft-beer.html'
s = HTMLSession()
drinklist = []

def request(url):
    r = s.get(url)
    r.html.render(sleep=6)
    return r.html.xpath('//*[@id="product-list"]', first=True)

def parse(products):
    for item in products.absolute_links:
        r = s.get(item)
        name = r.html.find('h2.title', first=True).text
        subtext = r.html.find('div.subtitle', first=True).text
        price = r.html.find('span.price', first=True).text

        try:
            rating = r.html.find('span.label-stars', first=True).text
        except:
            rating = 'none'
        if r.html.find('div.add-to-cart-container'):
            stock = 'in stock'
        else:
            stock = 'out of stock'
        drink = {
            'name': name,
            'subtext': subtext,
            'price': price,
            'rating': rating,
            'stock': stock
        }
        drinklist.append(drink)

def output():
    df = pd.DataFrame(drinklist)
    df.to_csv('beerwulf_drinklist.csv', index=False)
    print('Saved to CSV Files')

products = request('	https://tmpfiles.org/dl/234075/product-list-craft-beer.html')
parse(products)
output()

RuntimeError: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.